### Saving data in bigdataviewer h5 format using np2bdv library

BigStitcher in Fiji can read in affine transformations and apply it to the raw data without need for deskewing the data
The idea here is to 
* convert the raw data to h5 file
* generate affine transformations and save it in the bigdataviewer xml file


In [3]:
from skimage.io import imread
import pyclesperanto_prototype as cle
from pyclesperanto_prototype._tier8 import AffineTransform3D

import numpy as np

import npy2bdv


In [4]:
#Read lattice file
rbc = imread("../sample_data/RBC_lattice.tif")

voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3

deskewing_angle_in_degrees = 30.0

print("Shape of original image", rbc.shape )

Shape of original image (150, 118, 209)


The affine matrices in pyclesperanto are defined in the xyz format which is compatible with in Java. 
However, if you need to use the affine matrix in Python, you will have to swap the first and third rows so its in the zyx format

In [16]:

deskew_transform = AffineTransform3D()

deskew_transform._deskew_y(
    angle_in_degrees=deskewing_angle_in_degrees,
    voxel_size_x=voxel_size_x_in_microns,
    voxel_size_y=voxel_size_y_in_microns,
    voxel_size_z=voxel_size_z_in_microns
)
deskew_transform._matrix

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  8.66025404e-01,  2.06907682e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -5.00000000e-01,  1.11022302e-16,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [17]:
save_dir = "C:\\Users\\deepu\\\Documents\\"
fname = save_dir +"\\data.h5"
affine_transform = deskew_transform._matrix[:3]  #3x4 matrix
bdv_writer = npy2bdv.BdvWriter(fname)
bdv_writer.append_view(rbc, 
                       time=0,
                       channel=0,
                       m_affine=affine_transform,
                       name_affine="deskew_transform",
                       voxel_size_xyz = (voxel_size_x_in_microns,voxel_size_y_in_microns,voxel_size_z_in_microns),
                       voxel_units= "um",
                       calibration=(1, 1, voxel_size_y_in_microns/voxel_size_x_in_microns))

bdv_writer.write_xml()
bdv_writer.close()
print(f"Deskew affine transformed dataset in {fname}")

Deskew affine transformed dataset in C:\Users\deepu\\Documents\\data.h5


Example deskewing using pyclesperanto

In [5]:
#Perform deskewing on rbc image  
deskewed = cle.deskew_y(rbc, 
                        angle_in_degrees=deskewing_angle_in_degrees, 
                        voxel_size_x=voxel_size_x_in_microns, 
                        voxel_size_y=voxel_size_y_in_microns, 
                        voxel_size_z=voxel_size_z_in_microns)

print("Shape of deskewed image",deskewed.shape)

Shape of deskewed image (59, 413, 209)
